<a href="https://colab.research.google.com/github/afijithpv/Python-assignments-/blob/main/Designathon_project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import render_template, url_for, flash, redirect, request, session
from app import app, mysql
from werkzeug.security import generate_password_hash, check_password_hash
import MySQLdb.cursors

@app.route('/')
def home():
    cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    cursor.execute('SELECT * FROM recipes')
    recipes = cursor.fetchall()
    return render_template('home.html', recipes=recipes)

@app.route('/recipe/<int:recipe_id>', methods=['GET', 'POST'])
def recipe_detail(recipe_id):
    cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    cursor.execute('SELECT * FROM recipes WHERE id = %s', (recipe_id,))
    recipe = cursor.fetchone()
    cursor.execute('SELECT comments.comment, users.username, comments.date_posted FROM comments JOIN users ON comments.user_id = users.id WHERE recipe_id = %s', (recipe_id,))
    comments = cursor.fetchall()
    return render_template('recipe_detail.html', recipe=recipe, comments=comments)

@app.route('/submit_recipe', methods=['GET', 'POST'])
def submit_recipe():
    if 'user_id' not in session:
        flash('You need to log in to submit a recipe', 'danger')
        return redirect(url_for('login'))
    if request.method == 'POST':
        name = request.form['name']
        ingredients = request.form['ingredients']
        instructions = request.form['instructions']
        cooking_time = request.form['cooking_time']
        serving_size = request.form['serving_size']
        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
        cursor.execute('INSERT INTO recipes (name, ingredients, instructions, cooking_time, serving_size, user_id) VALUES (%s, %s, %s, %s, %s, %s)',
                       (name, ingredients, instructions, cooking_time, serving_size, session['user_id']))
        mysql.connection.commit()
        flash('Your recipe has been submitted!', 'success')
        return redirect(url_for('home'))
    return render_template('submit_recipe.html')

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        email = request.form['email']
        password = generate_password_hash(request.form['password'])
        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
        cursor.execute('INSERT INTO users (username, email, password) VALUES (%s, %s, %s)', (username, email, password))
        mysql.connection.commit()
        flash('Your account has been created! You can now log in', 'success')
        return redirect(url_for('login'))
    return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']
        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
        cursor.execute('SELECT * FROM users WHERE email = %s', (email,))
        user = cursor.fetchone()
        if user and check_password_hash(user['password'], password):
            session['user_id'] = user['id']
            session['username'] = user['username']
            flash('You have been logged in!', 'success')
            return redirect(url_for('home'))
        else:
            flash('Login Unsuccessful. Please check email and password', 'danger')
    return render_template('login.html')

@app.route('/logout')
def logout():
    session.pop('user_id', None)
    session.pop('username', None)
    flash('You have been logged out', 'success')
    return redirect(url_for('home'))


In [ ]:
from flask import Flask
from flask_mysqldb import MySQL

app = Flask(__name__)
app.config.from_object('config.Config')

mysql = MySQL(app)

from app import routes

In [ ]:
from app import app

if __name__ == '__main__':
    app.run(debug=True)